## Wharton - People Analytics Conference
#### Teach For America
**The data used for this project is confidential and owned by TFA so all outputs have been cleared out.**

### I. Case Questions

1. How do characteristics of the applicant relate to whether the candidate arrives for Day 1 training and completes the program through Year 2? From information available to TFA before the final interview, predict the candidate outcomes at Year 2.

2. In what way, if at all, is cost-of-living (as provided in the data) related to candidate decisions?

3. How could TFA refine who they invite to interview to identify those who, if admitted, would be highly likely to begin training and complete their corps?

4. Implications of your analysis: Based on your analysis, what recruiting initiatives would you recommend to help TFA maximize the probability of interviewed candidates attending training and matriculating through Year 2 while accounting for candidate quality and geographic need?

5. We have 95k who recieve an interview invite and then we have 57k who complete the interview. Do we know how many showed up to the interview of the people who recieved an interview invitation?

## II. Data Collection + Cleanup


In [ ]:
# Import libraries
import warnings
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import chi2_contingency
from scipy.stats import ks_2samp
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve 
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.inspection import permutation_importance
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.inspection import permutation_importance
from scipy.stats import mannwhitneyu
from collections import Counter
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Read in the data
df = pd.read_csv('processed_df.csv')
df_t = pd.read_csv('text_df.csv')

In [ ]:
# Check for missing data in df
print(df.isnull().sum())

### Bivariate Analysis

In [ ]:
# Stacked barcharts function for categorical variables
def stacked_barcharts(dataframe, columns, app_step, fig_title, fig_height, rel_freq=None, ncols=5):
    nrows = 1 + (len(columns) - 1) // ncols
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(14, nrows * fig_height))
    for i, column in enumerate(columns):
        if rel_freq == True:
            # calculate pivot table
            pivot = pd.crosstab(dataframe[app_step], dataframe[column])
            # divide by column sums to get frequency per column
            freq = pivot.div(pivot.sum())
            # display as stacked bar chart with 100%
            ax = axes[i // 5, i % 5] if nrows > 1 else axes[i]
            with warnings.catch_warnings():
                warnings.simplefilter('ignore')
                freq.transpose().plot(kind='bar', ax=ax, stacked=True, legend=False)
        elif rel_freq == False:
            # calculate pivot table
            pivot = pd.crosstab(dataframe[app_step], dataframe[column])
            ax = axes[i // 5, i % 5] if nrows > 1 else axes[i]
            with warnings.catch_warnings():
                warnings.simplefilter('ignore')
                pivot.transpose().plot(kind='bar', ax=ax, stacked=True, legend=False)
        else:
            print('Please specify rel_freq=True or rel_freq=False')
    for i in range(len(columns), nrows * 5):
        ax = axes[i // 5, i % 5] if nrows > 1 else axes[i]
        fig.delaxes(ax)
    fig.suptitle(fig_title)
    plt.tight_layout(2.5)

In [ ]:
print("2021 DATA:\n")
# Total number of candidates in Progress_5_Start_1stDay
total_1stDay = df.Progress_5_Start_1stDay.value_counts()
print('Total number of people at this stage: {:,}'.format(total_1stDay.sum()))
# Percent of people that started their first day (Progress_5_Start_1stDay == 1)
percent_1stDay = total_1stDay[1] / total_1stDay.sum() * 100
print('Percent of people that started their first day: {:.2f}% ({:,})'.format(percent_1stDay, total_1stDay[1]))
# Percent of people that did not start their first day (Progress_5_Start_1stDay == 0)
percent_not_1stDay = total_1stDay[0] / total_1stDay.sum() * 100
print('Percent of people that did not made it to this step: {:.2f}% ({:,})'.format(percent_not_1stDay, total_1stDay[0]))


In [ ]:
# Creaste new df with the people started their first day (Progress_5_Start_1stDay = 1)
df_1stDay = df[df['Progress_5_Start_1stDay'] == 1]

In [ ]:
# Stacked bar of Progress_5_Start_1stDay and Progress_6_Complete_2yrs
plt.close('all')
plt.figure(figsize=(4,3))
ax = plt.subplot()
df_counts = df_1stDay.groupby(['Progress_5_Start_1stDay', 'Progress_6_Complete_2yrs']).size().unstack()
df_counts.plot(kind='bar', stacked=True, ax=ax, legend=True)
plt.title('Start_1stDay vs Complete_2yrs')
plt.xlabel('Progress_5_Start_1stDay')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

df_1stDay_2yrs = df_1stDay[df_1stDay['Progress_6_Complete_2yrs'] == 1].shape[0] / total_1stDay[1] * 100
print('Out of the {:,} people that started their first day, {:.2f}% ({:,}) completed the 2 years program'.format(total_1stDay[1], df_1stDay_2yrs ,df_counts[1][1]))

In [ ]:
# Group columns based on their information
col_list_s = [column for column in df_1stDay.columns if len(df_1stDay[column].unique()) <= 10]
col_list_l = [column for column in df_1stDay.columns if len(df_1stDay[column].unique()) >= 10]
#print(col_list_s)
#print('\n')
#print(col_list_l)

cat_cols_s = ['career_level', 'UG_school_selectivity', 'UG_major_minor_STEM', 'UG_sports', 'UG_PellGrant', 'Leadership_role', 'family_responsibility', 'filled_cols_count', 'filled_cols_count_bin', 'steps_completed', 'prev_LIC_level', 'Ivy_league', 'app_number', 'high_profile']
cat_cols_l = ['Region']
num_cols = ['App_start_date', 'App_submit_date', 'ConfirmOffer_date', 'UG_GPA', 'Cost', 'app_completion_days', 'financial_gap','avg_dimension_score', 'offer_delay_days']

In [ ]:

plt.close('all')
plt.figure(figsize=(6, 5))
ax = plt.subplot()

df_counts = df_1stDay.groupby(['filled_cols_count_bin', 'Progress_6_Complete_2yrs']).size().unstack()

# Calculate row totals and sort the DataFrame by row totals in descending order
df_counts['Total'] = df_counts.sum(axis=1)
df_counts.sort_values('Total', ascending=False, inplace=True)
df_counts.drop(columns='Total', inplace=True)
# create frequency of df_counts
df_counts = df_counts.div(df_counts.sum(axis=1), axis=0)
df_counts.plot(kind='bar', stacked=True, ax=ax, legend=True)
# add x tick labels with "Complete application form for 1 and Incomplete application form for 0"
ax.set_xticklabels(['Complete application form', 'Incomplete application form'])
# rotate labels 0 degrees
plt.xticks(rotation=0)

plt.title('Complete Application Form vs Candidate Completed 2yrs')
plt.xlabel('Progress_6_Complete_2yrs')
plt.ylabel('Count')

plt.show()

In [ ]:

plt.close('all')
plt.figure(figsize=(6, 5))
ax = plt.subplot()

df_counts = df_1stDay.groupby(['high_profile', 'Progress_6_Complete_2yrs']).size().unstack()

# Calculate row totals and sort the DataFrame by row totals in descending order
df_counts['Total'] = df_counts.sum(axis=1)
df_counts.sort_values('Total', ascending=False, inplace=True)
df_counts.drop(columns='Total', inplace=True)
# create frequency of df_counts
df_counts = df_counts.div(df_counts.sum(axis=1), axis=0)
df_counts.plot(kind='bar', stacked=True, ax=ax, legend=True)
# add x tick labels with "Complete application form for 1 and Incomplete application form for 0"
ax.set_xticklabels([0, 1, 2, 3])
# rotate labels 0 degrees
plt.xticks(rotation=0)

plt.title('High Profile Candidate vs Candidate Completed 2yrs')
plt.xlabel('Progress_6_Complete_2yrs')
plt.ylabel('Count')

plt.show()

#### Categorical columns

In [ ]:
# Graph cat_cols_s by Progress_6_Complete_2yrs
stacked_barcharts(dataframe=df_1stDay, columns=cat_cols_s, app_step='Progress_6_Complete_2yrs', fig_title='Progress_6_Complete_2yrs', fig_height=3, rel_freq=True, ncols=5)

In [ ]:
plt.close('all')
plt.figure(figsize=(15, 5))
ax = plt.subplot()

df_counts = df_1stDay.groupby(['Region', 'Progress_6_Complete_2yrs']).size().unstack()
df_counts.rename(columns={0:'Incomplete', 1:'Complete'}, inplace=True)
# create frequency of df_counts
df_counts = df_counts.div(df_counts.sum(axis=1), axis=0)
df_counts.sort_values('Incomplete', ascending=False, inplace=True)
df_counts.plot(kind='bar', stacked=True, ax=ax, legend=True)

plt.title('Percent of candidates who completed the program by region')
plt.xlabel('Progress_6_Complete_2yrs')
plt.ylabel('Percentage of candidates [%]')

plt.show()

#### Numerical columns

In [ ]:
def plot_histograms(dataframe, column_list, graph_type='hist', density=None):
    nrows = len(column_list)
    fig, axes = plt.subplots(nrows=nrows, ncols=1, figsize=(10, 3 * nrows))
    for i, col in enumerate(column_list):
        if graph_type == 'hist':
            ax = axes[i]
            ax.hist(dataframe[dataframe['Progress_6_Complete_2yrs'] == 0][col], bins=20, density=density, alpha=0.5, label='Did not complete 2 years')
            ax.hist(dataframe[dataframe['Progress_6_Complete_2yrs'] == 1][col], bins=20, density=density, alpha=0.5, label='Completed 2 years')
            ax.set_title(f"{col} vs completed 2 years")
            ax.set_xlabel(col)
            ax.set_ylabel('Count')
            ax.legend()
        elif graph_type == 'kde':
            ax = axes[i]
            sns.kdeplot(dataframe[dataframe['Progress_6_Complete_2yrs'] == 0][col], ax=ax, label='Did not complete 2 years')
            sns.kdeplot(dataframe[dataframe['Progress_6_Complete_2yrs'] == 1][col], ax=ax, label='Completed 2 years')
            ax.set_title(f"{col} vs completed 2 years")
            ax.set_xlabel(col)
            ax.set_ylabel('Density')
            ax.legend()
        else:
            print("Invalid graph type")
    plt.tight_layout()
    plt.show()


In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 3))

sns.kdeplot(df_1stDay[df_1stDay['Progress_6_Complete_2yrs'] == 0]['Cost'], ax=ax, label='Did not complete 2 years')
sns.kdeplot(df_1stDay[df_1stDay['Progress_6_Complete_2yrs'] == 1]['Cost'], ax=ax, label='Completed 2 years')
ax.set_title('Avg. cost of living vs completed 2 years')
ax.set_xlabel('Average Cost of Living in Region [USD]')
ax.set_ylabel('Density')
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
num_cols = ['App_start_date', 'App_submit_date', 'ConfirmOffer_date', 'UG_GPA', 'Cost', 'app_completion_days', 'financial_gap','avg_dimension_score', 'offer_delay_days']
num_cols = ['UG_GPA', 'Cost', 'financial_gap','avg_dimension_score', 'App_start_date', 'App_submit_date', 'ConfirmOffer_date', 'app_completion_days', 'offer_delay_days']
plot_histograms(dataframe=df_1stDay, column_list=num_cols, graph_type='hist', density=True)


In [ ]:
# Correlation matrix
cor_df = df_1stDay.drop(columns=['variable', 'filled_cols_count_bin', 'family_responsibility', 'financial_gap', 'steps_completed', 'Progress_1_Invite_Intrvw', 'Progress_2_Complete_Intrvw', 'Progress_3_Accepted_toCorp', 'Progress_4_Confirm_Offer', 'Progress_5_Start_1stDay'])
progress_col = cor_df['Progress_6_Complete_2yrs']
cor_df.drop(columns=['Progress_6_Complete_2yrs'], inplace=True)
cor_df['Progress_6_Complete_2yrs'] = progress_col
# calculate correlation matrix
corr = np.abs(cor_df.corr())
fig, ax = plt.subplots(figsize=(8, 8))
cmap = sns.color_palette('magma')
sns.heatmap(corr, cmap=cmap, square=True)
plt.title('Correlation between numerical df: abs values')
plt.show()

#### Hypothesis testing

In [ ]:
# Run chi2 test on filled_cols_count_bin and Progress_6_Complete_2yrs
chi2, pval, dof, ex = chi2_contingency(pd.crosstab(df_1stDay['filled_cols_count_bin'], df_1stDay['Progress_6_Complete_2yrs']))
print('p-value:', pval)
print('\n')
print('Null: The variables are independent. No relationship exists.')
print('Alternative: A relationship between the variables exists.\n')
print('Pval is less than 0.05, so we reject the null hypothesis. There is a relationship between filled_cols_count_bin and Progress_6_Complete_2yrs')


In [ ]:
# Run Kolmogorov-Smirnov test on cost vs progress_6_complete_2yrs
from scipy.stats import ks_2samp
stat, pval = stats.ks_2samp(df_1stDay[df_1stDay.Progress_6_Complete_2yrs == 0].Cost, df_1stDay[df_1stDay.Progress_6_Complete_2yrs == 1].Cost)
print('p-value:', pval)
print('\n')
print('Null: The variables are independent. No relationship exists.')
print('Alternative: A relationship between the variables exists.\n')
print('Pval is less than 0.05, so we reject the null hypothesis. There is a relationship between filled_cols_count_bin and Progress_6_Complete_2yrs')

In [ ]:
# Run chi2 test on high_profile and Progress_6_Complete_2yrs
chi2, pval, dof, ex = chi2_contingency(pd.crosstab(df_1stDay['high_profile'], df_1stDay['Progress_6_Complete_2yrs']))
print('p-value:', pval)
print('\n')
print('Null: The variables are independent. No relationship exists.')
print('Alternative: A relationship between the variables exists.\n')
print('Pval is greater than 0.05, so we fail to reject the null hypothesis. There is no relationship between high_profile and Progress_6_Complete_2yrs')

#### Modeling

In [ ]:
# Filter people that started their first day in Progress_5_Start_1stDay
df = df[df['Progress_5_Start_1stDay'] == 1]

In [ ]:
# Check nulls in df and unique values in df of categories with 6 or less unique values
{column : ['Nulls: ' + str(df[column].isnull().sum()), 'Unique values: ' + str(df[column].nunique())] for column in df.columns}

In [ ]:
# Fill GPA missing values with the mean of the column
df['UG_GPA'].fillna(df['UG_GPA'].mean(), inplace=True)
# Fill region missing values with the mode of the column
#df['Region'].fillna(df['Region'].mode()[0], inplace=True)
# Fill cost missing values with the mean of the column
df['Cost'].fillna(df['Cost'].mean(), inplace=True)
# Fill financial gap missing values with the mean of the column
df['financial_gap'].fillna(df['financial_gap'].mean(), inplace=True)
# Fill avg_dimension_score missing values with the mean of the column
df['avg_dimension_score'].fillna(df['avg_dimension_score'].mean(), inplace=True)

In [ ]:
# One hot encode Region column
#df = pd.get_dummies(df, columns=['Region'], prefix=['Region'])

In [ ]:
# Create labels and features
labels = df['Progress_6_Complete_2yrs']
features = df.drop(['offer_delay_days', 'Leadership_role', 'Ivy_league', 'app_number', 'Region', 'avg_dimension_score', 'financial_gap', 'App_start_date', 'App_submit_date', 'ConfirmOffer_date', 'steps_completed', 'variable', 'Progress_1_Invite_Intrvw', 'Progress_2_Complete_Intrvw', 'Progress_3_Accepted_toCorp', 'Progress_4_Confirm_Offer', 'Progress_5_Start_1stDay', 'Progress_6_Complete_2yrs'], axis=1)

In [ ]:
{column : 'Nulls: ' + str(features[column].isnull().sum()) for column in features.columns }

In [ ]:
# Determine which features are relevant for training.
from sklearn.feature_selection import VarianceThreshold
threshold_n=0.95
sel = VarianceThreshold(threshold=(threshold_n* (1 - threshold_n) ))
sel_var=sel.fit_transform(features)
features[features.columns[sel.get_support(indices=True)]] 

In [ ]:
# Split training and test sets
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size = 0.20, random_state = 1)

#### Logistic Regression

In [ ]:
# Create and train the model with pipeline
lr_pipe = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000, class_weight='balanced'))
lr_pipe.fit(x_train, y_train)
# Predictions
lr_predictions = lr_pipe.predict(x_test)
# Confusion Matrix
cm = confusion_matrix(y_test, lr_predictions)
ax = sns.heatmap(cm, linewidth = 0.5, annot = True, cmap = 'Blues', fmt = 'g') 
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.ylabel('Predicted values')
plt.xlabel('Real values')
plt.show()
# Model score
print('Train Score: ', lr_pipe.score(x_train, y_train))
print('Test Score: ', lr_pipe.score(x_test, y_test))
# Classification Report
print(classification_report(y_test, lr_predictions))

In [ ]:
# Permutation importances
result = permutation_importance(lr_pipe, x_test, y_test, n_repeats=10, random_state=42, n_jobs=2)

sorted_importances_idx = result.importances_mean.argsort()
importances = pd.DataFrame(result.importances[sorted_importances_idx].T, columns=features.columns[sorted_importances_idx],)
# create figure and set size
#fig, ax = plt.subplots(figsize=(6, 15))
ax = importances.plot.box(vert=False, whis=10)
ax.set_title("Permutation Importances (test set)")
ax.axvline(x=0, color="k", linestyle="--")
ax.set_xlabel("Decrease in accuracy score")
ax.figure.tight_layout()

In [ ]:
# Graph importances from coefficients
lr = lr_pipe.steps[1][1]
importances = pd.DataFrame(data={
    'Attribute': x_train.columns,
    'Importance': lr.coef_[0]
})
importances = importances.sort_values(by='Importance', ascending=False)
# Plot results
plt.figure(figsize=(15, 6))
plt.bar(x=importances['Attribute'], height=importances['Importance'], color='dodgerblue')
plt.title('Feature importances obtained from coefficients')
plt.xticks(rotation='vertical')
plt.show()

#### Random Forests

In [ ]:
# Train random forests classifier
rf = RandomForestClassifier(n_estimators=300, max_depth=10, class_weight='balanced')
rf.fit(x_train, y_train)
# Predictions
rf_predictions = rf.predict(x_test)
# Confusion Matrix
cm = confusion_matrix(y_test, rf_predictions)
ax = sns.heatmap(cm, linewidth = 0.5, annot = True, cmap = 'Greens', fmt = 'g') 
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.ylabel('Predicted values')
plt.xlabel('Real values')
plt.show()
# Model score
print('Train Score: ', rf.score(x_train, y_train))
print('Test Score: ', rf.score(x_test, y_test))
# Classification Report
print(classification_report(y_test, rf_predictions))

In [ ]:
# Permutation importances
result = permutation_importance(rf, x_test, y_test, n_repeats=10, random_state=42, n_jobs=2)

sorted_importances_idx = result.importances_mean.argsort()
importances = pd.DataFrame(result.importances[sorted_importances_idx].T, columns=features.columns[sorted_importances_idx],)
# create figure and set size
ax = importances.plot.box(vert=False, whis=10)
ax.set_title("Permutation Importances (test set)")
ax.axvline(x=0, color="k", linestyle="--")
ax.set_xlabel("Decrease in accuracy score")
ax.figure.tight_layout()

In [ ]:
# Graph importances from coefficients
importances = pd.DataFrame(data={
    'Attribute': x_train.columns,
    'Importance': rf.feature_importances_
})
importances = importances.sort_values(by='Importance', ascending=False)
# Plot results
plt.figure(figsize=(15, 6))
plt.bar(x=importances['Attribute'], height=importances['Importance'], color='mediumseagreen')
plt.title('Feature importances obtained from coefficients')
plt.xticks(rotation='vertical')
plt.show()

*Note: The tendency of this approach is to inflate the importance of continuous features or high-cardinality categorical variables*

#### K-Neighbors

In [ ]:
# Create and train the model with pipeline
knn_pipe = make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors = 100))
knn_pipe.fit(x_train, y_train)
# Predictions
knn_predictions = knn_pipe.predict(x_test)
# Confusion Matrix
cm = confusion_matrix(y_test, knn_predictions)
ax = sns.heatmap(cm, linewidth = 0.5, annot = True, cmap = 'Reds', fmt = 'g') 
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.ylabel('Predicted values')
plt.xlabel('Real values')
plt.show()
# Model score
print('Train Score: ', knn_pipe.score(x_train, y_train))
print('Test Score: ', knn_pipe.score(x_test, y_test))
# Classification Report
print(classification_report(y_test, knn_predictions))

In [ ]:
# Permutation importances
result = permutation_importance(knn_pipe, x_test, y_test, n_repeats=10, random_state=42, n_jobs=2)

sorted_importances_idx = result.importances_mean.argsort()
importances = pd.DataFrame(result.importances[sorted_importances_idx].T, columns=features.columns[sorted_importances_idx],)
# create figure and set size
ax = importances.plot.box(vert=False, whis=10)
ax.set_title("Permutation Importances (test set)")
ax.axvline(x=0, color="k", linestyle="--")
ax.set_xlabel("Decrease in accuracy score")
ax.figure.tight_layout()